## Neuro-Fuzzy Rule Miner Demo

This Notebook serves as a simple demo of the classifying neuro-fuzzy rule miner using the example of the iris dataset.

Outline:
- About Neuro-Fuzziness
- Dataset
- Running the model
- Results

To run this project in Google Colab, either download this file and upload it manually or use the github option with the path: https://github.com/juelha/NeuroFuzzyRuleMiner/blob/demo/DEMO.ipynb.

In [1]:
# @title Run to Clone
!git clone --branch demo https://github.com/juelha/NeuroFuzzyRuleMiner.git
%cd NeuroFuzzyRuleMiner

fatal: destination path 'NeuroFuzzyRuleMiner' already exists and is not an empty directory.
/content/NeuroFuzzyRuleMiner


## About Neuro-Fuzziness <a class="anchor" id="neuro-fuzzy"></a>

#### What?
- neuro-fuzzy = models that adjust fuzzy sets and rules according to neural networks tuning techniques
- neuro-fuzzy system = when the model can act as a fuzzy system once it is trained

#### Why?
- Knowledge acquisition directly from data
- White box neural nets

#### How?

<img align="right" width="400" src="https://github.com/juelha/NeuroFuzzyRuleMiner/blob/demo/doc/figures/nf_annot.svg?raw=1" hspace="50">

The architecture is comprised of three layers: for fuzzification, for the IF-part of a rule and for the THEN-part of a rule.
The propagation of the input through the architecture is computed as follows:


**Fuzzification-Layer:**

The input vector $\vec{x}$ is fuzzified. Each input $x_i$ has its own set of membership functions $MF_i$, for example: $MF_i = (MF_{low}, MF_{medium}, MF_{high})$.  The outputs of this layer can be referred to as the degrees of membership, $\mu_{ij}$, of an input $x_i$ to a function $MF_{ij}$.
$$ \mu_{ij} = MF_{ij}(x_i) $$


**IF-Layer:**

The fuzzified inputs are combined via a T-norm operation, here multiplication. This represents the if-part of the fuzzy rule. The outputs of this layer is referred to as the rule strengths $\vec{R} = (r_1, r_2, ..., r_{m^n})$, where $\rho$ = (1, ..., $m^n$) for $n$ inputs and $m$ membership functions.
$$ R_\rho = \text{T-norm} (\mu_{ij}, \mu_{(i+1)j}, ..., \mu_{nm}) = \mu_{ij} * \mu_{(i+1)j} * ... * \mu_{nm} $$

**THEN-Layer:**

The rule strengths are mapped to classes by one-hot encoded class weights.
$$ y_\rho = R_\rho * class weight_\rho $$

## Dataset <a id="data"></a>

<img align="right" width="600" src="https://github.com/juelha/NeuroFuzzyRuleMiner/blob/demo/doc/figures/iris_scatter.png?raw=1" hspace="10">

The Iris dataset is a popular classification problem where four features were measured for the three iris species setosa, versicolor, virginica.

On the right is a scatter plot of the Iris Dataset. The first 50 samples belong to the species setosa, the samples from 50 to 100 to versicolor, and the samples from 100 to 150 to virginica.


### Running the model

<img align="right" width="500" src="https://github.com/juelha/NeuroFuzzyRuleMiner/blob/demo/doc/figures/repo_struct.svg?raw=1" hspace="10">


Each operation that is performed on the neuro-fuzzy architecture is implemented in a script, i.e. Builder, Trainer, Classifier, and Rule Miner.

**Builder:**
- initializes the free parameters: parameters of the membership functions, and
class weights

**Trainer:**
- trains the parameters of the membership functions with gradient descent

**Rule Miner:**
- extracts the IF-THEN rules from a trained neuro-fuzzy model  

**Classifier:**
- propagates a sample through the model and outputs the class with the highest activation


In [4]:
# imports
from model_pkg import *
from neurofuzzy_pkg import *

# hyper parameters
learning_rate = 1
n_epochs = 1

# parameters for running iris dataset
df_name = "iris"
fuzzy_labels = ["small" , "medium","high"]
lingusitic_output = ["Setosa", "Versicolour", "Virginica"]
n_participants = 4

MyModel = Model(DataPipeline(df_name),
                    MyArc(fuzzy_labels, n_participants, len(lingusitic_output)),
                    MyArcTrainer(n_epochs=n_epochs, learning_rate=learning_rate),
                    Builder(),
                    Classifier())
MyModel.build() # init MF params and class weights
MyModel.train()
print(MyModel.class_accuracy())


selecting: 100%|██████████| 81/81 [00:00<00:00, 18403.05it/s]


Build done


testing: 100%|██████████| 120/120 [00:00<00:00, 863.28it/s]


Epoch: 0 starting with 
             test accuracy 0.9333333333333333 
             train accuracy 0.95 
             test loss 0.1627985672936622 
             train loss 0.1650228362260136


testing: 100%|██████████| 30/30 [00:00<00:00, 1905.35it/s]


Training Loop completed


class testing: 100%|██████████| 150/150 [00:00<00:00, 3362.37it/s]


┌─────────────────────────────────────────────────────────────────┐
  Classifying Results: Accuracy total: 0.947, No. incorrect: 8 
└─────────────────────────────────────────────────────────────────┘

0.9466666666666667


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

### Results <a class="anchor" id="results"></a>

In [5]:
# @title Show rule table
# get save path

import os
import pandas as pd

file_name = 'iris_best__rules.csv'
relative_path =   '/results/iris'

save_path = os.getcwd() +  relative_path # https://stackoverflow.com/questions/39125532/file-does-not-exist-in-jupyter-notebook
full_path = os.path.join(save_path, file_name)
assert  os.path.exists(full_path), f'Path {relative_path}/{file_name} does not exist'
df = pd.read_csv(full_path).rename(columns={"Unnamed: 0": "Rule No."})
df

,Rule No.,sepal length,sepal width,petal length,petal width,Class,Activations
0,10,small,medium,small,small,Setosa,0.107836
1,41,medium,medium,medium,medium,Versicolour,0.083070
2,32,medium,small,medium,medium,Versicolour,0.042747
3,37,medium,medium,small,small,Setosa,0.026257
4,45,medium,medium,high,high,Virginica,0.025801
5,42,medium,medium,medium,high,Virginica,0.020032
6,72,high,medium,high,high,Virginica,0.015928
7,46,medium,high,small,small,Setosa,0.013719
8,5,small,small,medium,medium,Versicolour,0.011363
9,19,small,high,small,small,Setosa,0.010577


<center>
<img src="https://github.com/juelha/NeuroFuzzyRuleMiner/blob/demo/results/iris/figures/PerformanceMyArc.png?raw=1" width=500>
</center>